In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy as np
import pyttsx3
import openpyxl

# Load the trained model to classify signs
from keras.models import load_model
model = load_model('traffic_classifier.h5')

# Dictionary mapping class indices to class names
classes = {1: 'Speed limit (20km/h)', 2: 'Speed limit (30km/h)', 3: 'Speed limit (50km/h)', 4: 'Speed limit (60km/h)',
           5: 'Speed limit (70km/h)', 6: 'Speed limit (80km/h)', 7: 'End of speed limit (80km/h)',
           8: 'Speed limit (100km/h)', 9: 'Speed limit (120km/h)', 10: 'No passing',
           11: 'No passing veh over 3.5 tons', 12: 'Right-of-way at intersection', 13: 'Priority road', 14: 'Yield',
           15: 'Stop', 16: 'No vehicles', 17: 'Veh > 3.5 tons prohibited', 18: 'No entry', 19: 'General caution',
           20: 'Dangerous curve left', 21: 'Dangerous curve right', 22: 'Double curve', 23: 'Bumpy road',
           24: 'Slippery road', 25: 'Road narrows on the right', 26: 'Road work', 27: 'Traffic signals',
           28: 'Pedestrians', 29: 'Children crossing', 30: 'Bicycles crossing', 31: 'Beware of ice/snow',
           32: 'Wild animals crossing', 33: 'End speed + passing limits', 34: 'Turn right ahead',
           35: 'Turn left ahead', 36: 'Ahead only', 37: 'Go straight or right', 38: 'Go straight or left',
           39: 'Keep right', 40: 'Keep left', 41: 'Roundabout mandatory', 42: 'End of no passing',
           43: 'End no passing veh > 3.5 tons'}

# Initialize GUI
top = tk.Tk()
top.geometry('800x600')
top.title('Traffic sign classification')
top.configure(background='#CDCDCD')

label = Label(top, background='#CDCDCD', font=('arial', 15, 'bold'))
sign_image = Label(top)

engine = pyttsx3.init()


def classify(file_path):
    global label_packed
    image = Image.open(file_path)
    image = image.resize((30, 30))
    image = np.expand_dims(image, axis=0)
    pred_prob = model.predict(image)[0]
    pred_class = np.argmax(pred_prob)
    sign = classes[pred_class + 1]
    print(sign)
    label.configure(foreground='#011638', text=sign)
    speak(sign)
    # Ask for feedback
    ask_feedback(file_path, sign)


def speak(text):
    engine.say(text)
    engine.runAndWait()


def ask_feedback(file_path, sign):
    feedback_window = tk.Toplevel(top)
    feedback_window.title("Feedback")
    feedback_window.geometry("300x250")

    feedback_label = Label(feedback_window, text=f"Was the classification of '{sign}' correct?", font=('arial', 12))
    feedback_label.pack(pady=10)

    feedback_var = IntVar()
    feedback_var.set(0)  # Default: No feedback
    feedback_radio_correct = Radiobutton(feedback_window, text="Correct", variable=feedback_var, value=1)
    feedback_radio_incorrect = Radiobutton(feedback_window, text="Incorrect", variable=feedback_var, value=2)
    feedback_radio_correct.pack()
    feedback_radio_incorrect.pack()

    submit_feedback_btn = Button(feedback_window, text="Submit",
                                 command=lambda: save_feedback(file_path, sign, feedback_var.get(), feedback_window))
    submit_feedback_btn.pack(pady=10)


def save_feedback(file_path, sign, feedback, feedback_window):
    feedback_text = "Correct" if feedback == 1 else "Incorrect"
    try:
        workbook = openpyxl.load_workbook('traffic_signs.xlsx')
        sheet = workbook.active
        max_row = sheet.max_row

        # Add image to excel sheet
        img = openpyxl.drawing.image.Image(file_path)
        img.anchor = f'A{max_row + 3}'
        sheet.add_image(img)

        # Add sign to excel sheet
        sheet[f'B{max_row + 3}'] = sign

        # Add feedback to excel sheet
        sheet[f'C{max_row + 3}'] = feedback_text

        workbook.save('traffic_signs.xlsx')
        feedback_window.destroy()
    except Exception as e:
        print(e)


def show_classify_button(file_path):
    classify_b = Button(top, text="Classify Image", command=lambda: classify(file_path), padx=10, pady=5)
    classify_b.configure(background='#364156', foreground='white', font=('arial', 10, 'bold'))
    classify_b.place(relx=0.79, rely=0.46)


def upload_image():
    try:
        file_path = filedialog.askopenfilename()
        uploaded = Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width() / 2.25), (top.winfo_height() / 2.25)))
        im = ImageTk.PhotoImage(uploaded)

        sign_image.configure(image=im)
        sign_image.image = im
        label.configure(text='')
        show_classify_button(file_path)
    except Exception as e:
        print(e)


upload = Button(top, text="Upload an image", command=upload_image, padx=10, pady=5)
upload.configure(background='#364156', foreground='white', font=('arial', 10, 'bold'))

upload.pack(side=BOTTOM, pady=50)
sign_image.pack(side=BOTTOM, expand=True)
label.pack(side=BOTTOM, expand=True)
heading = Label(top, text="Know Your Traffic Sign", pady=20, font=('arial', 20, 'bold'))
heading.configure(background='#CDCDCD', foreground='#364156')
heading.pack()
top.mainloop()


1/1 [==============================] - 4s 4s/step
Speed limit (30km/h)
